# Issuer Agent

In [37]:
import requests
import json
import qrcode
import uuid
import re

In [38]:
# Set the URL for the API endpoint
# Need to specify the host as host.docker.internal as the Prism Agents and the Prism Playground are running in two isolated docker environments
base_url = "http://host.docker.internal:8080"

headers = {'Content-Type': 'application/json', 'accept': 'application/json'}

## Connections

In [39]:
path = '/prism-agent/connections'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

{
  "contents": [],
  "kind": "Collection",
  "self": "/collections"
}


### Create a connection record containing the invitation
```bash
curl -X 'POST' \
	'http://localhost:8080/prism-agent/connections' \
	-H 'Content-Type: application/json' \
	-d '{
		"label": "Connect with Alice"
		}' | jq
```

In [40]:
path = '/prism-agent/connections'
data = {"label": "Connect with Alice"}
# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 201:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-201 status code")

{
  "connectionId": "8f339aae-5e59-4f36-a77e-0f31ab58e6f4",
  "createdAt": "2022-12-14T18:40:37.592636Z",
  "invitation": {
    "from": "did:peer:2.Ez6LSkDER4ZHScADDoWsdCEpnxTwNsaUPhYk295idMLByvzFC.Vz6MkfyzkcimreZ5E5YAs3Z6Y4vC2MWYGX9EPWd99iQFG3C3Q.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
    "id": "8f339aae-5e59-4f36-a77e-0f31ab58e6f4",
    "invitationUrl": "https://domain.com/path?_oob=eyJpZCI6IjhmMzM5YWFlLTVlNTktNGYzNi1hNzdlLTBmMzFhYjU4ZTZmNCIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNrREVSNFpIU2NBRERvV3NkQ0VwbnhUd05zYVVQaFlrMjk1aWRNTEJ5dnpGQy5WejZNa2Z5emtjaW1yZVo1RTVZQXMzWjZZNHZDMk1XWUdYOUVQV2Q5OWlRRkczQzNRLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qZ3dPREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBi

### Send invitation to Holder

In [41]:
invitation = json.loads(response.content.decode())
invitationUrl = invitation['invitation']['invitationUrl']
print(invitationUrl)
invitationUrlbase64 = re.sub("^.*_oob=", "", invitationUrl)
print(invitationUrlbase64)
# invitationUrlbase64 = invitation['invitation']['invitationUrl'].split('=')[1:5]
connection_id = invitation['connectionId']
%store invitationUrlbase64


https://domain.com/path?_oob=eyJpZCI6IjhmMzM5YWFlLTVlNTktNGYzNi1hNzdlLTBmMzFhYjU4ZTZmNCIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNrREVSNFpIU2NBRERvV3NkQ0VwbnhUd05zYVVQaFlrMjk1aWRNTEJ5dnpGQy5WejZNa2Z5emtjaW1yZVo1RTVZQXMzWjZZNHZDMk1XWUdYOUVQV2Q5OWlRRkczQzNRLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qZ3dPREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWVuIHR3byBwZWVycyIsImFjY2VwdCI6W119fQ==
eyJpZCI6IjhmMzM5YWFlLTVlNTktNGYzNi1hNzdlLTBmMzFhYjU4ZTZmNCIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNrREVSNFpIU2NBRERvV3NkQ0VwbnhUd05zYVVQaFlrMjk1aWRNTEJ5dnpGQy5WejZNa2Z5emtjaW1yZVo1RTVZQXMzWjZZNHZDMk1XWUdYOUVQV2Q5OWlRRkczQzNRLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qZ3dPREF2Wkdsa1kyOX

### Generate QR code from invitation

In [ ]:
# Link for connection invitation
input_data = invitationUrl
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=10,
        border=5)
qr.add_data(input_data)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img.save('issuer_agent_invite_QRcode.png')

from IPython.display import Image
Image(width=400, filename='./issuer_agent_invite_QRcode.png')

### Retrieving the list of connections
```bash
curl -X 'GET' 'http://localhost:8080/prism-agent/connections' | jq
```

In [45]:

path = '/prism-agent//connections/' + connection_id

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    their_did = loaded_json['theirDid']
    STATE = loaded_json['state']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

{
  "connectionId": "8f339aae-5e59-4f36-a77e-0f31ab58e6f4",
  "createdAt": "2022-12-14T18:40:37Z",
  "invitation": {
    "from": "did:peer:2.Ez6LSkDER4ZHScADDoWsdCEpnxTwNsaUPhYk295idMLByvzFC.Vz6MkfyzkcimreZ5E5YAs3Z6Y4vC2MWYGX9EPWd99iQFG3C3Q.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
    "id": "8f339aae-5e59-4f36-a77e-0f31ab58e6f4",
    "invitationUrl": "https://domain.com/path?_oob=eyJpZCI6IjhmMzM5YWFlLTVlNTktNGYzNi1hNzdlLTBmMzFhYjU4ZTZmNCIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNrREVSNFpIU2NBRERvV3NkQ0VwbnhUd05zYVVQaFlrMjk1aWRNTEJ5dnpGQy5WejZNa2Z5emtjaW1yZVo1RTVZQXMzWjZZNHZDMk1XWUdYOUVQV2Q5OWlRRkczQzNRLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qZ3dPREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWV

In [59]:
import time
from termcolor import colored,cprint

path = '/prism-agent//connections/' + connection_id

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'ConnectionResponseSent':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['state']
        their_did = loaded_json['theirDid']
        print(colored("ConnectionId {0} is not in active state yet. State: {1}".format(connection_id,STATE), "yellow", attrs=["bold"]))

    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))


Current state for ConnectionId 8f339aae-5e59-4f36-a77e-0f31ab58e6f4 is ConnectionResponseSent
ConnectionId: 8f339aae-5e59-4f36-a77e-0f31ab58e6f4 is now active. Continue with notebook


## Head over to the Issuer Notebook and accept connection

## Issue Credential to Holder

### Initiate a new issue credential flow

Replace `{SUBJECT_ID}` with the DID of the holder displayed at startup in the his Prism Agent console logs
```bash
curl -X 'POST' \
  'http://localhost:8080/prism-agent/issue-credentials/credential-offers' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
      "schemaId": "schema:1234",
      "subjectId": "{SUBJECT_ID}",
      "validityPeriod": 3600,
      "automaticIssuance": false,
      "awaitConfirmation": false,
      "claims": {
        "firstname": "Alice",
        "lastname": "Wonderland",
        "birthdate": "01/01/2000"
      }
	}' | jq
```

In [48]:
# subject_id = 'did:peer:2.Ez6LSsyLCEveJGeF9jWGkBCG6wvfzXcNgu5A8PAiKp3NzvPdb.Vz6MkuZxR1VDeXiJ9uL9Uuh7Rh58DGpYw3abdtacLv8KdL2sP.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbS8iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19'
path = '/prism-agent/issue-credentials/credential-offers'
data = {
      "schemaId": "schema:1234",
      "subjectId": their_did,
      "validityPeriod": 3600,
      "automaticIssuance": False,
      "awaitConfirmation": False,
      "claims": {
        "firstname": "Alice",
        "lastname": "Wonderland",
        "birthdate": "01/01/2000"
      }
    }

# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 201:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-201 status code")
    print(response.content)

{
  "automaticIssuance": false,
  "awaitConfirmation": false,
  "claims": {
    "birthdate": "01/01/2000",
    "firstname": "Alice",
    "lastname": "Wonderland"
  },
  "createdAt": "2022-12-14T18:42:11.188892Z",
  "protocolState": "OfferPending",
  "recordId": "51dc271c-4abf-472a-89b9-253d2256c61a",
  "role": "Issuer",
  "schemaId": "schema:1234",
  "subjectId": "did:peer:2.Ez6LSdEYSAtkCArFgUXXY5g4RhXP23ej1Ynu8UejzjU9YrX3E.Vz6Mksj7wesBWcbKV7so6ipvaKYY9wse6aXc7rEYxWLXdJ8zw.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
  "validityPeriod": 3600.0
}


In [49]:
credential_record_id = loaded_json['recordId']
%store credential_record_id

Stored 'credential_record_id' (str)


In [50]:
path = '/prism-agent/issue-credentials/records'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

{
  "count": 1,
  "items": [
    {
      "automaticIssuance": false,
      "awaitConfirmation": false,
      "claims": {
        "birthdate": "01/01/2000",
        "firstname": "Alice",
        "lastname": "Wonderland"
      },
      "createdAt": "2022-12-14T18:42:11Z",
      "protocolState": "OfferPending",
      "recordId": "51dc271c-4abf-472a-89b9-253d2256c61a",
      "role": "Issuer",
      "schemaId": "schema:1234",
      "subjectId": "did:peer:2.Ez6LSdEYSAtkCArFgUXXY5g4RhXP23ej1Ynu8UejzjU9YrX3E.Vz6Mksj7wesBWcbKV7so6ipvaKYY9wse6aXc7rEYxWLXdJ8zw.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
      "validityPeriod": 3600.0
    }
  ],
  "limit": 0,
  "offset": 0
}


### - Issuing the credential

> Replace `{RECORD_ID}` with the UUID of the record from the previous list
```bash
curl -X 'POST' 'http://localhost:8080/prism-agent/issue-credentials/records/{RECORD_ID}/issue-credential' | jq
```

In [51]:
path = '/prism-agent/issue-credentials/records/' + credential_record_id + '/issue-credential'
# Make the request
response = requests.post(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

{
  "automaticIssuance": false,
  "awaitConfirmation": false,
  "claims": {
    "birthdate": "01/01/2000",
    "firstname": "Alice",
    "lastname": "Wonderland"
  },
  "createdAt": "2022-12-14T18:42:11Z",
  "protocolState": "CredentialPending",
  "recordId": "51dc271c-4abf-472a-89b9-253d2256c61a",
  "role": "Issuer",
  "schemaId": "schema:1234",
  "subjectId": "did:peer:2.Ez6LSdEYSAtkCArFgUXXY5g4RhXP23ej1Ynu8UejzjU9YrX3E.Vz6Mksj7wesBWcbKV7so6ipvaKYY9wse6aXc7rEYxWLXdJ8zw.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
  "updatedAt": "2022-12-14T18:42:26Z",
  "validityPeriod": 3600.0
}


In [58]:
path = '/prism-agent/issue-credentials/records'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

{
  "count": 1,
  "items": [
    {
      "automaticIssuance": false,
      "awaitConfirmation": false,
      "claims": {
        "birthdate": "01/01/2000",
        "firstname": "Alice",
        "lastname": "Wonderland"
      },
      "createdAt": "2022-12-14T18:42:11Z",
      "jwtCredential": "ImV5SjBlWEFpT2lKS1YxUWlMQ0poYkdjaU9pSkZVekkxTmlKOS5leUpwYzNNaU9pSmthV1E2Y0hKcGMyMDZZekF4T0dJMU1HUXRZalpsWXkwMFlUSXlMV0l4TXpVdE5qVmtNbVZsTWpCaU16RTFJaXdpYzNWaUlqb2laR2xrT25CbFpYSTZNaTVGZWpaTVUyUkZXVk5CZEd0RFFYSkdaMVZZV0ZrMVp6UlNhRmhRTWpObGFqRlpiblU0VldWcWVtcFZPVmx5V0RORkxsWjZOazFyYzJvM2QyVnpRbGRqWWt0V04zTnZObWx3ZG1GTFdWazVkM05sTm1GWVl6ZHlSVmw0VjB4WVpFbzRlbmN1VTJWNVNqQkphbTlwV2tjd2FVeERTbnBKYW05cFlVaFNNR05FYjNaTU1taDJZek5SZFZwSE9XcGhNbFo1VEcxc2RXUkhWbmxpYlVaelQycG5kMDlVUVhaYVIyeHJXVEk1ZEdKVFNYTkpia2xwVDJ4MFpFeERTbWhKYW5CaVNXMVNjRnBIVG5aaVZ6QjJaR3BKYVZoWU1DSXNJblpqSWpwN0ltTnlaV1JsYm5ScFlXeFRkV0pxWldOMElqcDdJbUpwY25Sb1pHRjBaU0k2SWpBeEx6QXhMekl3TURBaUxDSm1hWEp6ZEc1aGJXVWlPaUpCYkdsalpTSXNJbXhoYzNSdVlXM

## DIDs

### Create a new unpublised DID stored in Prism Agent
```bash
curl --location --request POST 'http://localhost:8080/prism-agent/did-registrar/dids' \
  --header 'Content-Type: application/json' \
  --header 'Accept: application/json' \
  --data-raw '{
    "documentTemplate": {
      "publicKeys": [
        {
          "id": "auth0",
          "purpose": "authentication"
        }
      ],
      "services": []
    }
  }'
```

In [53]:
path = '/prism-agent/did-registrar/dids'
data = {
    "documentTemplate": {
      "publicKeys": [
        {
          "id": "auth0",
          "purpose": "authentication"
        }
      ],
      "services": []
    }
  }

# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

{
  "longFormDid": "did:prism:b1430d510aa6e8b56a508dfeef13821166c376a98286998003834dc07062be3e:Cr0BCroBEloKBWF1dGgwEARCTwoJc2VjcDI1NmsxEiCCtQdcUQbxTDrkpmFi1Dk7NUZ7zPeNZNUtGFoLrvDJsxogcjFna4jsdgKRiFhoF0w3fgxoUkpC4eoWimqJPHZSRx4SXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIgWfoZ3b_t5E51C1zVCI3Wasa-rUY47PDmSYQDWZh9t-8aICn0-0PYz2-aTQKh8ShrUta2UACq4Mr883zIDZqRMHvq"
}


### Lists all the DIDs in Prism Agent
```bash
curl --location --request GET 'http://localhost:8080/prism-agent/did-registrar/dids' \
  --header 'Accept: application/json'
```



In [54]:
path = '/prism-agent/did-registrar/dids'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    DID_REF = loaded_json[0]['longFormDid']
    DID_REF_SHORT = loaded_json[0]['did']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

[
  {
    "did": "did:prism:b1430d510aa6e8b56a508dfeef13821166c376a98286998003834dc07062be3e",
    "longFormDid": "did:prism:b1430d510aa6e8b56a508dfeef13821166c376a98286998003834dc07062be3e:Cr0BCroBEloKBWF1dGgwEARCTwoJc2VjcDI1NmsxEiCCtQdcUQbxTDrkpmFi1Dk7NUZ7zPeNZNUtGFoLrvDJsxogcjFna4jsdgKRiFhoF0w3fgxoUkpC4eoWimqJPHZSRx4SXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIgWfoZ3b_t5E51C1zVCI3Wasa-rUY47PDmSYQDWZh9t-8aICn0-0PYz2-aTQKh8ShrUta2UACq4Mr883zIDZqRMHvq",
    "status": "CREATED"
  }
]


### Publishes the DID in Prism Agent to the blockchain
Replace `DID_REF` by the DID on Prism Agent that should be published
```bash
curl --location --request POST 'http://localhost:8080/prism-agent/did-registrar/dids/{DID_REF}/publications' \
--header 'Accept: application/json'
```

In [55]:
path = '/prism-agent/did-registrar/dids/' + DID_REF + '/publications'

# Make the request
response = requests.post(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 202:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-202 status code")
    print(response.content)

Error: The API returned a non-202 status code
b'{"detail":"OperationError(DLTProxyError(io.grpc.StatusRuntimeException: UNAVAILABLE))","instance":"error-instance","status":500,"title":"error-title","type":"error-type"}'


### Resolves the DID document of Prism DID
Replace `DID_REF` by the DID on Prism Agent that should be resolved
```bash
curl --location --request GET 'http://localhost:8080/prism-agent/dids/{DID_REF}' \
--header 'Accept: application/json'
```

In [56]:
path = '/prism-agent/dids/' + DID_REF

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

Error: The API returned a non-200 status code
b'{"detail":"InternalError","instance":"error-instance","status":500,"title":"internalError","type":"error-type"}'
